In [2]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import time

In [17]:
def day_difference(date_str1, date_str2):
    start_date = datetime.strptime(date_str1, '%d %m %Y')
    end_date = datetime.strptime(date_str2, '%d %b %Y')
    date_difference = end_date - start_date
    return date_difference.days

2

In [41]:
def get_airport_arrivals(airport_code,year,month,day):

    #Scraping from a few elements before the date selected to get full arrivals list.
    date_time = datetime(year,month,day,23,0)
    converted_date = date_time - timedelta(days=1)
    startDate = time.mktime(converted_date.timetuple())
    
    flights = []
    pageNum = 0
    #Generate URL for scraping
    while(True):

        url = 'https://www.avionio.com/en/airport/' + airport_code + '/departures' + '?ts=' + str(int(startDate)) + '000&page=' + str(pageNum)

        #add format to string for date
        
        #Get HTML from URL
        res = requests.get(url)
        htmlData = res.content
        parsedData = BeautifulSoup(htmlData, 'html.parser')
        
        #Rows with departure data
        info_table = parsedData.find_all('tr',class_='tt-row')
    
        for row in info_table:
            if 'tt-child' in row['class']:  
                continue
            
            flight = {
                'time': row.find('td',class_='tt-t').text.strip(),
                'date': row.find('td',class_='tt-d').text.strip() + ' 2024',
                'iata': row.find('td',class_='tt-i').text.strip(),
                'destination': row.find('td',class_='tt-ap').text.strip(),
                'flight': row.find('td',class_='tt-f').text.strip(),
                'airline': row.find('td',class_='tt-al').text.strip(),
                'status': row.find('td',class_='tt-s').text.strip()
            }
        	
            if (day_difference(f'{day} {month} {year}',row.find('td',class_='tt-d').text.strip() + ' 2024')):
                return flights
            

            flights.append(flight)
        pageNum += 1

flights = get_airport_arrivals('bcn', 2024, 1, 23)

{'time': '03:15', 'date': '24 Jan 2024', 'iata': 'PMI', 'destination': 'Palma de Mallorca', 'flight': 'WT109', 'airline': 'Swiftair', 'status': 'Unknown'}


In [43]:
len(flights)

377